In [ ]:
#NB: Kaggle requires phone verification to use the internet or a GPU. If you haven't done that yet, the cell below will fail
#    This code is only here to check that your internet is enabled. It doesn't do anything else.
#    Here's a help thread on getting your phone number verified: https://www.kaggle.com/product-feedback/135367

import socket,warnings
try:
    socket.setdefaulttimeout(1)
    socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect(('1.1.1.1', 53))
except socket.error as ex: raise Exception("STOP: No internet. Click '>|' in top right and set 'Internet' switch to on")

In [ ]:
# It's a good idea to ensure you're running the latest version of any libraries you need.
# `!pip install -Uqq <libraries>` upgrades to the latest version of <libraries>
# NB: You can safely ignore any warnings or errors pip spits out about running as root or incompatibilities
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -Uqq fastai duckduckgo_search

In [ ]:
# from duckduckgo_search import ddg_images
from duckduckgo_search import DDGS
from fastcore.all import *

ddgs = DDGS()

def search_images(term, max_images=30):
    print(f"Searching for '{term}'")
#     return L(ddg_images(term, max_results=max_images)).itemgot('image')
    return L(ddgs.images(keywords=term, max_results=max_images)).itemgot('image')

In [ ]:
#NB: `search_images` depends on duckduckgo.com, which doesn't always return correct responses.
#    If you get a JSON error, just try running it again (it may take a couple of tries).
urls = search_images('black labrador photos', max_images=1)
urls[0]

In [ ]:
#downloads black lab photos and then sizes it. we need to confirm this is the right photo
from fastdownload import download_url
dest = 'blacklab.jpg'
download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

In [1]:
#downloads an image of a poodle, and opens it to confirm
download_url(search_images('poodles', max_images=1)[0], 'poodle.jpg', show_progress=False)
Image.open('poodle.jpg').to_thumb(256,256)
#confirmed search works

NameError: name 'download_url' is not defined

In [ ]:
#gather more pictures of black labs vs other dogs and save to folder to train and test later with different conditions (puppy/adult, colors)
searches = 'poodles','blacklab'
path = Path('blacklab_or_not')
from time import sleep

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} photo'))
    sleep(10)  # Pause between searches to avoid over-loading server
    download_images(dest, urls=search_images(f'{o} puppy photo'))
    sleep(10)
    download_images(dest, urls=search_images(f'{o} adult photo'))
    sleep(10)
    download_images(dest, urls=search_images(f'{o} black photo'))
    sleep(10)
    resize_images(path/o, max_size=400, dest=path/o)

In [2]:
#start to train model
#filter out any failed images
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

NameError: name 'verify_images' is not defined

To train a model, we'll need `DataLoaders`, which is an object that contains a *training set* (the images used to create a model) and a *validation set* (the images used to check the accuracy of a model -- not used during training). In `fastai` we can create that easily using a `DataBlock`, and view sample images from it:

In [3]:
#using DataLoaders (object containing a training set + validation set)
#DataBlock is a fastAI component to use dataloader + view sample images via show_batch
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=6)

NameError: name 'DataBlock' is not defined

In [5]:
#fastai comes with fine_time() method that automatically uses best practices for a pre-trained model
#resnet is the fastest widely used model
#fine-tuning lets us use a pre-trained model and adjust weights to recognize our dataset. resnet18 was trained to recognize photos from 1000 categories
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

NameError: name 'vision_learner' is not defined

In [ ]:
## Step 3: Use our dog-trained model

In [6]:
#model predicting if photo in validation set is a dog
is_dog,_,probs = learn.predict(PILImage.create('blacklab.jpg'))
print(f"This is a: {is_dog}.")
print(f"Probability it's a black lab: {probs[0]:.4f}")


NameError: name 'learn' is not defined